In [38]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
from torch import nn as nn
from torchvision.datasets import Food101
from trak import TRAKer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [2]:
processor = AutoImageProcessor.from_pretrained("microsoft/resnet-18")
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-18")

preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [59]:
def preprocess_image(image):
    image = transforms.functional.pil_to_tensor(image)
    processed_image = (processor.preprocess(image)["pixel_values"][0])
    return torch.from_numpy(processed_image)

train_dataset = Food101("data/food-101", split="train", transform=preprocess_image, download=True)
test_dataset = Food101("data/food-101", split="test", transform=preprocess_image, download=True)

In [60]:
train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

for X, y in train_dl:
    break

X.shape, y.shape

(torch.Size([64, 3, 224, 224]), torch.Size([64]))

In [66]:
model

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

In [67]:
num_classes = 101

model.classifier = nn.Sequential(
                    nn.Flatten(start_dim=1, end_dim=-1),
                    nn.Linear(in_features=512, out_features=num_classes))

model.num_labels = 101

In [62]:
def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False

    for param in model.classifier.parameters():
        param.requires_grad = True

In [68]:
freeze_model(model)

In [83]:
def train(model, epochs: int, optimizer):
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(epochs):
        running_loss = 0.0

        model.train()

        train_correct = 0
        train_outputs = 0

        for i, data in enumerate(tqdm(train_dl), 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs.logits, labels)

            loss.backward()
            optimizer.step()

            train_correct += (torch.argmax(outputs.logits, dim=-1) == labels).sum().item()
            train_outputs += outputs.logits.shape[0]

            running_loss += loss

        model.eval()
        total_correct = 0
        total_outputs = 0

        with torch.no_grad():
            for i, data in enumerate(tqdm(test_dl), 0):
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                
                outputs = model(images)
                correct = (torch.argmax(outputs.logits, dim=-1) == labels).sum().item()

                total_correct += correct
                total_outputs += outputs.logits.shape[0]

        print(f"[Epoch {epoch + 1}] Loss: {running_loss / i:.3f}, Train Acc: {train_correct/train_outputs:.3f}," +
              f"Valid Acc: {total_correct/total_outputs:.3f}")

In [85]:
optimizer = torch.optim.Adam(model.parameters())
num_epochs = 4

train(model, num_epochs, optimizer)  

100%|█████████████████████████████████████████| 395/395 [19:04<00:00,  2.90s/it]


[Epoch 1] Loss: 5.936, Train Acc: 0.513,Valid Acc: 0.554


100%|█████████████████████████████████████████| 395/395 [18:40<00:00,  2.84s/it]


[Epoch 2] Loss: 5.574, Train Acc: 0.536,Valid Acc: 0.568


100%|█████████████████████████████████████████| 395/395 [17:42<00:00,  2.69s/it]


[Epoch 3] Loss: 5.403, Train Acc: 0.549,Valid Acc: 0.571


100%|█████████████████████████████████████████| 395/395 [17:38<00:00,  2.68s/it]

[Epoch 4] Loss: 5.296, Train Acc: 0.555,Valid Acc: 0.576


In [104]:
# TODO finetune the model on food101 dataset -> use TRAK -> finetune again on the base model -> look at the results

In [105]:
traker = TRAKer(model, task="image_classification", train_set_size=len(dataset))

ERROR:TRAK:Could not use CudaProjector.
Reason: No module named 'fast_jl'
ERROR:TRAK:Defaulting to BasicProjector.
INFO:STORE:No existing model IDs in C:\Users\kamil\OneDrive\Pulpit\przedmioty\semestr 8\automating science\trak-for-automating-science\trak_results.
INFO:STORE:No existing TRAK scores in C:\Users\kamil\OneDrive\Pulpit\przedmioty\semestr 8\automating science\trak-for-automating-science\trak_results.
